# Working with Databases in Python using `sqlite` and `sqlalchemy`

## SQLite using `sqlite3` CLI

In [1]:
!which sqlite3

/Users/neilbardhan/opt/anaconda3/bin/sqlite3


In [2]:
import os
os.chdir('..')

### List all tables

In [3]:
!sqlite3 data/db/books.db .tables

author  book  


### `SELECT` Statements

In [4]:
!sqlite3 data/db/books.db 'SELECT * FROM author;'

1|Pearl|Buck
2|Mark|Twain
3|Tom|Clancy
4|Isaac|Asimov


In [5]:
!sqlite3 data/db/books.db 'SELECT * FROM book;'

1|Foundation|4
2|The Good Earth|1
3|The Good Earth|1
4|Adventures of Huckleberry Finn|2
5|Life on the Mississippi|2
6|The Hunt for Red October|3


### Working with `.sql` files

`.sql` files allow us to persist queries that we would like to run against a database.

In [6]:
QUERY = """
SELECT
a.first_name, a.last_name, b.title AS book_title
FROM author a JOIN book b
ON b.author_id = a.author_id
ORDER BY a.last_name asc;
"""

In [7]:
with open("code/book_author_names.sql", "w") as code_file:
    code_file.write(QUERY)

In [8]:
!cat code/book_author_names.sql


SELECT
a.first_name, a.last_name, b.title AS book_title
FROM author a JOIN book b
ON b.author_id = a.author_id
ORDER BY a.last_name asc;


In [9]:
!sqlite3 data/db/books.db < code/book_author_names.sql

Isaac|Asimov|Foundation
Pearl|Buck|The Good Earth
Pearl|Buck|The Good Earth
Tom|Clancy|The Hunt for Red October
Mark|Twain|Adventures of Huckleberry Finn
Mark|Twain|Life on the Mississippi


In [10]:
!cat code/mark_twain.sql

SELECT b.title
FROM author a JOIN book b
ON b.author_id = a.author_id
WHERE a.first_name = 'Mark' AND a.last_name = 'Twain';

In [11]:
!sqlite3 data/db/books.db < code/mark_twain.sql

Adventures of Huckleberry Finn
Life on the Mississippi


## SQLite using the `sqlite` module

In [12]:
import sqlite3
import pandas as pd

In [13]:
DB_PATH = os.getcwd() + '/data/db/'
DB_FILE = 'books.db'

### Create a connection to the database

In [14]:
conn = sqlite3.connect(DB_PATH + DB_FILE)
conn.total_changes

0

The `sqlite3.connect()` function - 

  - Returns a `Connection` object that is used to interact with the SQLite database in `books.db`. 
  - If `books.db` had not existed prior to running the above code, it would have been created in the `DB_PATH` folder location.
  
`conn.total_changes` is the total number of database rows that have been changed by `conn`. Since we have not executed any SQL commands yet, 0 total_changes is correct.

### Run queries using the `cursor`

In [15]:
cursor = conn.cursor()

`connection.cursor()` returns a `Cursor` object. `Cursor` objects allow us to send SQL statements to a SQLite database using `cursor.execute()`.

In [16]:
print(dir(cursor))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'close', 'connection', 'description', 'execute', 'executemany', 'executescript', 'fetchall', 'fetchmany', 'fetchone', 'lastrowid', 'row_factory', 'rowcount', 'setinputsizes', 'setoutputsize']


In [17]:
select_query = """
SELECT * FROM book;
"""
rows = cursor.execute(select_query).fetchall()
rows

[(1, 'Foundation', 4),
 (2, 'The Good Earth', 1),
 (3, 'The Good Earth', 1),
 (4, 'Adventures of Huckleberry Finn', 2),
 (5, 'Life on the Mississippi', 2),
 (6, 'The Hunt for Red October', 3)]

In [18]:
rows_df = pd.read_sql_query(select_query, conn)
rows_df

,book_id,title,author_id
0,1,Foundation,4
1,2,The Good Earth,1
2,3,The Good Earth,1
3,4,Adventures of Huckleberry Finn,2
4,5,Life on the Mississippi,2
5,6,The Hunt for Red October,3


In [19]:
book_authors_df = pd.read_sql_query(QUERY, conn)
book_authors_df

,first_name,last_name,book_title
0,Isaac,Asimov,Foundation
1,Pearl,Buck,The Good Earth
2,Pearl,Buck,The Good Earth
3,Tom,Clancy,The Hunt for Red October
4,Mark,Twain,Adventures of Huckleberry Finn
5,Mark,Twain,Life on the Mississippi


In [20]:
conn.close()